In [1]:
# imports
import requests
import json
import os
import platform
import time
import math
import datetime
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [2]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

In [3]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [4]:
# scan text for predefined terms

text = 'We use CNN for anomaly detection. As Convolutional Neural Networks are great for ML.'

ml_terms = pd.read_csv('../data/patterns/ml_terms.csv')
ml_slugs = ml_terms['Slug'].tolist()
ml_slugs = [x for x in ml_slugs if str(x) != 'nan']
ml_terms = ml_terms['Term'].tolist()

ml_libs = pd.read_csv('../data/patterns/ml_libraries.csv')
ml_libs = ml_libs['Python Package'].tolist()

def match_text(haystack, needles, toLower = False, unique = True):
    
    if toLower == True:
        haystack = haystack.lower()
        needles = [x.lower() for x in needles]
    
    if unique == True:
        matches = {x for x in needles if x in haystack}
        matches = list(matches)
    else:
        matches = [x for x in needles if x in haystack]
    
    return matches

#ml_slugs, ml_terms, ml_libs, match_text(haystack, needles, toLower = False, unique = True)
needles = {
    'ml_slugs': ml_slugs,
    'ml_terms': ml_terms,
    'ml_libs': ml_libs,
}

print(match_text(text, ml_terms, True))

['convolutional neural network', 'neural network', 'anomaly detection']


In [5]:
# get file modifictaion date
# https://stackoverflow.com/questions/237079/how-to-get-file-creation-modification-date-times-in-python

def creation_date(path_to_file, datetime = True):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        timestamp = os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            timestamp = stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            timestamp = stat.st_mtime
        
    if datetime == True:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))

    return timestamp
    
folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
notebook = 'notebook_02.html'
kernel = 'kernel.html'
print(creation_date(folder_base+folder+notebook))

2020-12-12 16:08:03


In [64]:
# scrape competition

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
dataset = 'dataset.html'

def scrape_competition(html):
    soup = BeautifulSoup(html, 'html.parser')
    meta = {}
    
    meta['title'] = soup.find('h1', class_="competition-header__title").text.strip()
    meta['subtitle'] = soup.find('h2', class_="competition-header__subtitle").text.strip()
    meta['type'] = soup.find('p', class_="competition-header__classification-text").text.strip()
    meta['organisation'] = soup.find('span', class_="competition-header__organization-name")
    if meta['organisation'] == None:
        meta['organisation'] = ''
    else:
        meta['organisation'] = meta['organisation'].text.strip()
    temp = soup.find_all('li', class_="horizontal-list-item horizontal-list-item--bullet horizontal-list-item--default")
    for item in temp:
        #print(item.text)
        if 'team' in item.text:
            meta['teams'] = item.text.replace('teams','').replace('team','').strip()
        if 'ago' in item.text:
            #item = BeautifulSoup(item, 'html.parser')
            item = item.select('li>span>span')[0]
            meta['date_closed'] = item.get('title')
    
    meta['description'] = soup.find('div', class_="competition-overview__content").text.strip()
    meta['tags'] = [x.text.strip() for x in soup.find_all('span', class_="CategoryButton_CategoryName-sc-c10946 jFsDhg")]
    
    
    

    return meta

html = load_data(folder_base+folder+dataset)
if 'file not found' in html:
    print(html)
meta = scrape_competition(html)
print(meta)
#store_data(links, folder_base+folder+out, toJson=True)

{'title': 'Lyft 3D Object Detection for Autonomous Vehicles', 'subtitle': 'Can you advance the state of the art in 3D object detection?', 'type': 'Featured prediction Competition', 'organisation': 'Lyft', 'teams': '547', 'date_closed': 'Wed Nov 13 2019 00:59:00 GMT+0100 (Mitteleuropäische Normalzeit)', 'description': 'Self-driving technology presents a rare opportunity to improve the quality of life in many of our communities. Avoidable collisions, single-occupant commuters, and vehicle emissions are choking cities, while infrastructure strains under rapid urban growth. Autonomous vehicles are expected to redefine transportation and unlock a myriad of societal, environmental, and economic benefits. You can apply your data analysis skills in this competition to advance the state of self-driving technology.\nLyft, whose mission is to improve people’s lives with the world’s best transportation, is investing in the future of self-driving vehicles. Level 5, their self-driving division, is w

In [70]:
# iterate competions

url = url = 'https://www.kaggle.com/c/'
folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
dataset = 'dataset.html'
file_out = 'meta.json'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', i, len(folders))
    #print(folder)
    #print('###', i, os.path.join(folder_base,folder,subfolder))
    path = os.path.join(folder_base,folder,dataset)
    i += 1

    if os.path.exists(path):
        print(path)
        html = load_data(path)
        if 'file not found' in html:
            print(html)
        meta = scrape_competition(html)
        meta['link'] = url+folder
        #print(meta)
        store_data(meta, os.path.join(folder_base,folder,file_out), toJson=True)
            
    if quit!=0 and i>quit:
        break

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', j, 'items')

folder: 419
subfolder: 0 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-competition-spring-2015\dataset.html
subfolder: 1 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-spring-20152\dataset.html
subfolder: 2 419
../data/repositories/kaggle/competitions/c/20-newsgroups-ciphertext-challenge\dataset.html
subfolder: 3 419
../data/repositories/kaggle/competitions/c/3d-object-detection-for-autonomous-vehicles\dataset.html
subfolder: 4 419
../data/repositories/kaggle/competitions/c/abstraction-and-reasoning-challenge\dataset.html
subfolder: 5 419
../data/repositories/kaggle/competitions/c/accelerometer-biometric-competition\dataset.html
subfolder: 6 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-big\dataset.html
subfolder: 7 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-small\dataset.html
subfolder: 8 419
../data/repositories/kaggle/competitions/c/acquire-valued-shoppers-challenge\dataset.

subfolder: 80 419
../data/repositories/kaggle/competitions/c/data-science-bowl-2018\dataset.html
subfolder: 81 419
../data/repositories/kaggle/competitions/c/data-science-bowl-2019\dataset.html
subfolder: 82 419
../data/repositories/kaggle/competitions/c/data-science-for-good-careervillage\dataset.html
subfolder: 83 419
../data/repositories/kaggle/competitions/c/data-science-for-good-city-of-los-angeles\dataset.html
subfolder: 84 419
../data/repositories/kaggle/competitions/c/data-science-london-scikit-learn\dataset.html
subfolder: 85 419
../data/repositories/kaggle/competitions/c/datasciencebowl\dataset.html
subfolder: 86 419
../data/repositories/kaggle/competitions/c/dato-native\dataset.html
subfolder: 87 419
../data/repositories/kaggle/competitions/c/decoding-the-human-brain\dataset.html
subfolder: 88 419
../data/repositories/kaggle/competitions/c/deepfake-detection-challenge\dataset.html
subfolder: 89 419
../data/repositories/kaggle/competitions/c/deloitte-churn-prediction\dataset.

subfolder: 164 419
../data/repositories/kaggle/competitions/c/home-depot-product-search-relevance\dataset.html
subfolder: 165 419
../data/repositories/kaggle/competitions/c/homesite-quote-conversion\dataset.html
subfolder: 166 419
../data/repositories/kaggle/competitions/c/hospital\dataset.html
subfolder: 167 419
../data/repositories/kaggle/competitions/c/how-much-did-it-rain\dataset.html
subfolder: 168 419
../data/repositories/kaggle/competitions/c/how-much-did-it-rain-ii\dataset.html
subfolder: 169 419
../data/repositories/kaggle/competitions/c/human-protein-atlas-image-classification\dataset.html
subfolder: 170 419
../data/repositories/kaggle/competitions/c/humpback-whale-identification\dataset.html
subfolder: 171 419
../data/repositories/kaggle/competitions/c/icdar2013-gender-prediction-from-handwriting\dataset.html
subfolder: 172 419
../data/repositories/kaggle/competitions/c/icdar2013-stroke-recovery-from-offline-data\dataset.html
subfolder: 173 419
../data/repositories/kaggle/co

subfolder: 245 419
../data/repositories/kaggle/competitions/c/mens-machine-learning-competition-2019\dataset.html
subfolder: 246 419
../data/repositories/kaggle/competitions/c/mercari-price-suggestion-challenge\dataset.html
subfolder: 247 419
../data/repositories/kaggle/competitions/c/mercedes-benz-greener-manufacturing\dataset.html
subfolder: 248 419
../data/repositories/kaggle/competitions/c/MerckActivity\dataset.html
subfolder: 249 419
../data/repositories/kaggle/competitions/c/microsoft-malware-prediction\dataset.html
subfolder: 250 419
../data/repositories/kaggle/competitions/c/mlsp-2013-birds\dataset.html
subfolder: 251 419
../data/repositories/kaggle/competitions/c/mlsp-2014-mri\dataset.html
subfolder: 252 419
../data/repositories/kaggle/competitions/c/movie-review-sentiment-analysis-kernels-only\dataset.html
subfolder: 253 419
../data/repositories/kaggle/competitions/c/msdchallenge\dataset.html
subfolder: 254 419
../data/repositories/kaggle/competitions/c/msk-redefining-cancer-

subfolder: 323 419
../data/repositories/kaggle/competitions/c/rsna-str-pulmonary-embolism-detection\dataset.html
subfolder: 324 419
../data/repositories/kaggle/competitions/c/RTA\dataset.html
subfolder: 325 419
../data/repositories/kaggle/competitions/c/RxVolumePrediction\dataset.html
subfolder: 326 419
../data/repositories/kaggle/competitions/c/santa-2019-revenge-of-the-accountants\dataset.html
subfolder: 327 419
../data/repositories/kaggle/competitions/c/santa-gift-matching\dataset.html
subfolder: 328 419
../data/repositories/kaggle/competitions/c/santa-workshop-tour-2019\dataset.html
subfolder: 329 419
../data/repositories/kaggle/competitions/c/santander-customer-satisfaction\dataset.html
subfolder: 330 419
../data/repositories/kaggle/competitions/c/santander-customer-transaction-prediction\dataset.html
subfolder: 331 419
../data/repositories/kaggle/competitions/c/santander-product-recommendation\dataset.html
subfolder: 332 419
../data/repositories/kaggle/competitions/c/santander-va

subfolder: 402 419
../data/repositories/kaggle/competitions/c/whats-cooking-kernels-only\dataset.html
subfolder: 403 419
../data/repositories/kaggle/competitions/c/WIC2011\dataset.html
subfolder: 404 419
../data/repositories/kaggle/competitions/c/wikichallenge\dataset.html
subfolder: 405 419
../data/repositories/kaggle/competitions/c/wise-2014\dataset.html
subfolder: 406 419
../data/repositories/kaggle/competitions/c/womens-machine-learning-competition-2018\dataset.html
subfolder: 407 419
../data/repositories/kaggle/competitions/c/womens-machine-learning-competition-2019\dataset.html
subfolder: 408 419
../data/repositories/kaggle/competitions/c/word2vec-nlp-tutorial\dataset.html
subfolder: 409 419
../data/repositories/kaggle/competitions/c/worldcup2010\dataset.html
subfolder: 410 419
../data/repositories/kaggle/competitions/c/worldcupconf\dataset.html
subfolder: 411 419
../data/repositories/kaggle/competitions/c/yandex-personalized-web-search-challenge\dataset.html
subfolder: 412 419
.

In [73]:
# collect all meta.json into a single csv

folder_base = '../data/repositories/kaggle/competitions/c/'
file_meta = 'meta.json'
fp_csv = '../data/database/kaggle_competitions.csv'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()
df = pd.DataFrame()

runtime_start = time.time()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', i, len(folders))
    #print(folder)
    #print('###', i, os.path.join(folder_base,folder,subfolder))
    path = os.path.join(folder_base,folder,file_meta)
    i += 1

    if os.path.exists(path):
        print(path)
        data = load_data(path, fromJson=True)
        if 'file not found' in data:
            print(html)
            
        data['description'] = data['description'].replace('\n', ' ').replace('\r', '').replace('¶', '').strip()

        # date (ignoring GMT+x)
        date_time_str = data['date_closed'].split('GMT')
        date_time_str = date_time_str[0].strip()
        date_time_obj = datetime.datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')
        data['date_closed'] = date_time_obj
        
        df = df.append(data, ignore_index=True)
            
    if quit!=0 and i>quit:
        break
        
runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', j, 'items')
print(df.shape)
print(df.head())
        
# drop columns
#df.drop(columns=['author', 'submission'], inplace=True)

df.to_csv(fp_csv, sep=';', index=False)

folder: 419
subfolder: 0 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-competition-spring-2015\meta.json
subfolder: 1 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-spring-20152\meta.json
subfolder: 2 419
../data/repositories/kaggle/competitions/c/20-newsgroups-ciphertext-challenge\meta.json
subfolder: 3 419
../data/repositories/kaggle/competitions/c/3d-object-detection-for-autonomous-vehicles\meta.json
subfolder: 4 419
../data/repositories/kaggle/competitions/c/abstraction-and-reasoning-challenge\meta.json
subfolder: 5 419
../data/repositories/kaggle/competitions/c/accelerometer-biometric-competition\meta.json
subfolder: 6 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-big\meta.json
subfolder: 7 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-small\meta.json
subfolder: 8 419
../data/repositories/kaggle/competitions/c/acquire-valued-shoppers-challenge\meta.json
subfolder: 9 419
../da

../data/repositories/kaggle/competitions/c/deloitte-western-australia-rental-prices\meta.json
subfolder: 91 419
../data/repositories/kaggle/competitions/c/demand-forecasting-kernels-only\meta.json
subfolder: 92 419
../data/repositories/kaggle/competitions/c/denoising-dirty-documents\meta.json
subfolder: 93 419
../data/repositories/kaggle/competitions/c/detecting-insults-in-social-commentary\meta.json
subfolder: 94 419
../data/repositories/kaggle/competitions/c/diabetic-retinopathy-detection\meta.json
subfolder: 95 419
../data/repositories/kaggle/competitions/c/dog-breed-identification\meta.json
subfolder: 96 419
../data/repositories/kaggle/competitions/c/dogs-vs-cats\meta.json
subfolder: 97 419
../data/repositories/kaggle/competitions/c/dogs-vs-cats-redux-kernels-edition\meta.json
subfolder: 98 419
../data/repositories/kaggle/competitions/c/donorschoose-application-screening\meta.json
subfolder: 99 419
../data/repositories/kaggle/competitions/c/dont-call-me-turkey\meta.json
subfolder: 

../data/repositories/kaggle/competitions/c/imaterialist-challenge-FGVC2017\meta.json
subfolder: 178 419
../data/repositories/kaggle/competitions/c/imaterialist-challenge-furniture-2018\meta.json
subfolder: 179 419
../data/repositories/kaggle/competitions/c/imaterialist-fashion-2019-FGVC6\meta.json
subfolder: 180 419
../data/repositories/kaggle/competitions/c/imaterialist-fashion-2020-fgvc7\meta.json
subfolder: 181 419
../data/repositories/kaggle/competitions/c/imet-2019-fgvc6\meta.json
subfolder: 182 419
../data/repositories/kaggle/competitions/c/imet-2020-fgvc7\meta.json
subfolder: 183 419
../data/repositories/kaggle/competitions/c/inaturalist-2018\meta.json
subfolder: 184 419
../data/repositories/kaggle/competitions/c/inaturalist-2019-fgvc6\meta.json
subfolder: 185 419
../data/repositories/kaggle/competitions/c/inaturalist-challenge-at-fgvc-2017\meta.json
subfolder: 186 419
../data/repositories/kaggle/competitions/c/inclusive-images-challenge\meta.json
subfolder: 187 419
../data/repo

../data/repositories/kaggle/competitions/c/NFL-Punt-Analytics-Competition\meta.json
subfolder: 262 419
../data/repositories/kaggle/competitions/c/nips-2017-defense-against-adversarial-attack\meta.json
subfolder: 263 419
../data/repositories/kaggle/competitions/c/nips-2017-non-targeted-adversarial-attack\meta.json
subfolder: 264 419
../data/repositories/kaggle/competitions/c/nips-2017-targeted-adversarial-attack\meta.json
subfolder: 265 419
../data/repositories/kaggle/competitions/c/noaa-fisheries-steller-sea-lion-population-count\meta.json
subfolder: 266 419
../data/repositories/kaggle/competitions/c/noaa-right-whale-recognition\meta.json
subfolder: 267 419
../data/repositories/kaggle/competitions/c/nomad2018-predict-transparent-conductors\meta.json
subfolder: 268 419
../data/repositories/kaggle/competitions/c/nyc-taxi-trip-duration\meta.json
subfolder: 269 419
../data/repositories/kaggle/competitions/c/online-sales\meta.json
subfolder: 270 419
../data/repositories/kaggle/competitions/

subfolder: 346 419
../data/repositories/kaggle/competitions/c/siim-isic-melanoma-classification\meta.json
subfolder: 347 419
../data/repositories/kaggle/competitions/c/socialNetwork\meta.json
subfolder: 348 419
../data/repositories/kaggle/competitions/c/sp-society-camera-model-identification\meta.json
subfolder: 349 419
../data/repositories/kaggle/competitions/c/spooky-author-identification\meta.json
subfolder: 350 419
../data/repositories/kaggle/competitions/c/springleaf-marketing-response\meta.json
subfolder: 351 419
../data/repositories/kaggle/competitions/c/stanford-covid-vaccine\meta.json
subfolder: 352 419
../data/repositories/kaggle/competitions/c/state-farm-distracted-driver-detection\meta.json
subfolder: 353 419
../data/repositories/kaggle/competitions/c/statoil-iceberg-classifier-challenge\meta.json
subfolder: 354 419
../data/repositories/kaggle/competitions/c/stayalert\meta.json
subfolder: 355 419
../data/repositories/kaggle/competitions/c/street-view-getting-started-with-ju

In [6]:
# scrape notebook content

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
notebook = 'notebook_02.html'

def scrape_notebook_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    meta = {}
    
    meta['author'] = soup.find('a', class_="sc-paWCZ jvKDQp").get('href').replace('/','')
    meta['title'] = soup.find('a', class_="KernelViewerContext_KernelTitle-sc-rdaqnd").text   
    meta['type'] = soup.find('span', class_="KernelViewerContext_KernelTypeInfo-sc-1l6fza6 kqxzvL").text.replace('using data from','').strip()
    meta['sources'] = soup.find('a', class_="KernelViewerContext_DataSourceUrl-sc-1dm3ij9 lpoMHV").text.strip()
    meta['tags'] = soup.find('span', class_="KernelViewerContext_CategoriesWrapper-sc-8yrjj NgcTE").text.split('·')
    meta['tags'] = list(filter(None, meta['tags']))
    meta['votes'] = soup.find('span', class_="vote-button__vote-count").text
    meta['submission'] = soup.select('div.kernel-code-pane__submission-info-content')
    if len(meta['submission']) > 0:
        meta['submission'] = meta['submission'][0].text
    #meta['votes'] = soup.find('span', class_="vote-button__vote-count").text
    meta['license'] = soup.select('div.kernel-code-pane__subtitle>a')
    if len(meta['license']) > 0:
        meta['license'] = meta['license'][0].text
    
    score = soup.select('div.kernel-code-pane__submission-score-value')
    if len(score) > 0:
        meta['score_private'] = score[0].text
        meta['score_public'] = score[1].text
    
    
    temp = soup.find('span', class_="KernelViewerContext_KernelSubtitle-sc-rltxca esPWpV").select('span') #.text.split('·')
    #print (temp)
    for item in temp:
        #print(item)
        if 'views' in item.text:
            meta['views'] = int(''.join(filter(str.isdigit, item.text)))
        if 'GMT' in item.get('title', 'nan'):
            meta['date'] = item.get('title')
    
    return meta

html = load_data(folder_base+folder+notebook)
if 'file not found' in html:
    print(html)
meta = scrape_notebook_content(html)
print(meta)
#store_data(links, folder_base+folder+out, toJson=True)

{'author': 'asimandia', 'title': 'lyft3d inference kernel', 'type': 'Python notebook', 'sources': 'multiple data sources', 'tags': ['gpu'], 'votes': '38', 'submission': 'Best Submission SuccessfulSubmitted by [ods.ai] blonde & asimandia in Kitti lands a year ago', 'license': 'Apache 2.0', 'score_private': '0.040', 'score_public': '0.040', 'views': 2595, 'date': 'Tue Oct 29 2019 16:27:40 GMT+0100 (Mitteleuropäische Normalzeit)'}


In [7]:
# scrape kernel content

folder_base = '../data/repositories/kaggle/competitions/c/'
folder1 = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
folder2 = 'airbnb-recruiting-new-user-bookings/notebooks/datadave/ndcg-score-r/'
notebook = 'notebook_02.html'
kernel = 'kernel.html'

def scrape_kernel_content(html, needles, snippet = False):
    soup = BeautifulSoup(html, 'html.parser')
    
    if snippet == True:
        code = soup.find('div', {"id": "code"}).text
    else:
        code = soup.find('body').text
    #print(code)
    
    meta = {}
    
    for key, value in needles.items():
        meta[key] = match_text(code, value, True)
        
    meta['description'] = soup.select('div.text_cell_render')
    #print(meta['description'])
    if len(meta['description']) > 0:
        meta['description'] = meta['description'][0].text.strip()
    else:
        meta['description'] = ''
    
    return meta

# test for non-kernel-file ('code' embedded in notebook.html)
html = load_data(folder_base+folder2+notebook)
if 'file not found' in html:
    print(html)
meta = scrape_kernel_content(html, needles)
print(meta)

# test for kernel-file ('code' in kernel.html)
html = load_data(folder_base+folder1+kernel)
if 'file not found' in html:
    print(html)
meta = scrape_kernel_content(html, needles)
print(meta)


#store_data(links, folder_base+folder+out, toJson=True)

{'ml_slugs': ['nlp', 'nn', 'ner', 'ml', 'ai'], 'ml_terms': ['rank', 'predict', 'deep learning', 'training data', 'classification', 'filter', 'train'], 'ml_libs': [], 'description': ''}
{'ml_slugs': ['dl', 'cv', 'nn', 'ann', 'ner', 'rl', 'ai'], 'ml_terms': ['epoch', 'convolutional neural network', 'recommend', 'layer', 'neural network', 'test data', 'predict', 'label', 'u-net', 'image segmentation', 'loss', 'model', 'activation function', 'filter', 'train', 'relu'], 'ml_libs': ['pytorch'], 'description': "Please check out Guido's excellent kernel here. In this kernel i show how to perform inference on test set using the trained model.\nI just added RaDAM optimzer and got some better score.\nYou can find the BEV of the test set here.\nUpdates:¶\nCorrected yaw calculation\nUsed category height information"}


In [8]:
# iterate all folders and compose results into meta.json

url = 'https://www.kaggle.com/'
folder_base = '../data/repositories/kaggle/competitions/'
file_notebook = 'notebook_02.html'
file_kernel = 'kernel.html'
file_out = 'meta.json'

skip = True # if true skip meta collection / set to false to recreate {file_out} from scratch
quit = 10000 # quit after n files processed
breakOnError = False

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0
j = 0

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', len(folders))
    for subfolder in subfolders:
        print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,subfolder, 'notebooks/')
        i += 1
        
        if os.path.exists(path):
            projects = os.listdir(path)

            for author in projects:
                #print('author:', author)
                items = os.listdir(os.path.join(folder_base,folder,subfolder, 'notebooks/', author))

                for notebook in items:
                    j+=1
                    print(' - ', j, 'author:', author, '/ notebook:', notebook)
                    path_notebook = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_notebook)
                    path_kernel = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_kernel)
                    path_out = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_out)
                    
                    meta = {}
                    
                    if skip and os.path.isfile(path_out):
                        print('skipped')
                            
                    else:
                        try:
                            # get meta-data
                            if os.path.isfile(path_notebook):
                                #print('notebook found')
                                meta['scraped_at'] = creation_date(path_notebook)
                                meta['link'] = f'{url}{author}/{notebook}'
                                meta['reference'] = f'{url}{folder}/{subfolder}'
                                html = load_data(path_notebook, fromJson=False)
                                if 'file not found' in html:
                                    print('notebook not found')
                                    break
                                meta.update(scrape_notebook_content(html))

                            # get meta-data
                            if os.path.isfile(path_kernel):
                                #print('kernel found')
                                html = load_data(path_kernel, fromJson=False)
                                if 'file not found' in html:
                                    print('kernel not found')
                                    break
                                meta.update(scrape_kernel_content(html, needles))
                            else:
                                #print('kernel not found')
                                html = load_data(path_notebook, fromJson=False)
                                if 'file not found' in html:
                                    print('notebook not found')
                                    break
                                meta.update(scrape_kernel_content(html, needles))

                            #print(meta)
                            store_data(meta, path_out, toJson=True)
                            
                        except Exception as e:
                            print("Oops!", e.__class__, "occurred.")
                            print(e)
                            if breakOnError:
                                break
                
                if j>quit:
                    break
            if j>quit:
                break
    if j>quit:
        break

folder: 1
subfolder: 1
### 0 ../data/repositories/kaggle/competitions/c\15-071x-the-analytics-edge-competition-spring-2015
### 1 ../data/repositories/kaggle/competitions/c\15-071x-the-analytics-edge-spring-20152
### 2 ../data/repositories/kaggle/competitions/c\20-newsgroups-ciphertext-challenge
 -  1 author: a45632 / notebook: classification-tfidf-svm-2-0
skipped
 -  2 author: amansohane / notebook: level-3-with-partial-deciphering-0-94-level-3
skipped
 -  3 author: ananthu017 / notebook: classification-tfidf-logistic
skipped
 -  4 author: ashishpatel26 / notebook: attension-layer-basic-for-nlp
skipped
 -  5 author: ashishpatel26 / notebook: beginner-to-intermediate-nlp-tutorial
skipped
 -  6 author: ashishpatel26 / notebook: everything-you-want-to-know-about-20-ngctc
skipped
 -  7 author: ashishpatel26 / notebook: stratified-kfold-hyperparameter-tuning
skipped
 -  8 author: delayedkarma / notebook: some-basic-explorations-lgb-baseline
skipped
 -  9 author: interneuron / notebook: diff

 -  109 author: inversion / notebook: run-length-decoding-quick-start
skipped
 -  110 author: julian3833 / notebook: 2-understanding-and-plotting-rle-bounding-boxes
skipped
 -  111 author: julian3833 / notebook: 4-exploring-public-models
skipped
 -  112 author: kmader / notebook: baseline-u-net-model-part-1
skipped
 -  113 author: kmader / notebook: from-trained-u-net-to-submission-part-2
skipped
 -  114 author: kmader / notebook: transfer-learning-for-boat-or-no-boat
skipped
 -  115 author: kotarojp / notebook: first-step-for-submission-u-net-tta
skipped
 -  116 author: leighplt / notebook: pytorch-tutorial-dataset-data-preparetion-stage
skipped
 -  117 author: meaninglesslives / notebook: airbus-ship-detection-data-visualization
skipped
 -  118 author: npatta01 / notebook: naive-model
skipped
 -  119 author: rackovic1994 / notebook: convolutional-neural-network
skipped
 -  120 author: voglinio / notebook: from-masks-to-bounding-boxes
skipped
 -  121 author: windsurfer / notebook: bas

 -  230 author: abhishek / notebook: beating-the-benchmark-1
skipped
 -  231 author: adamwong / notebook: script-aminos-glm-logit
skipped
 -  232 author: alledluviette / notebook: ctr-test
skipped
 -  233 author: brianbai / notebook: inspect-tables-with-pandas
skipped
 -  234 author: btwardow / notebook: r-dplyr-sqlite
skipped
 -  235 author: datayo / notebook: python-data-explorer
skipped
 -  236 author: jeffmoser / notebook: database-schema
skipped
 -  237 author: jeffmoser / notebook: sample-rows-from-each-table
skipped
 -  238 author: olivermeyfarth / notebook: logistic-regression-on-histctr
skipped
 -  239 author: rahulpatel11315 / notebook: read-data-from-tsv-file-using-pandas-dataframe
skipped
 -  240 author: roenbaeck / notebook: number-of-records-in-every-table
skipped
 -  241 author: rootua / notebook: apache-spark-scala-logistic-regression
skipped
 -  242 author: ryutek / notebook: join-test
skipped
 -  243 author: satomacoto / notebook: logistic-regression
skipped
 -  244 a

notebook not found
 -  364 author: navinmundhra / notebook: birdcall-starter-tablular-image-data
skipped
 -  365 author: parulpandey / notebook: eda-and-audio-processing-with-python
skipped
 -  366 author: pavansanagapati / notebook: birds-sounds-eda-spotify-urban-sound-eda
skipped
 -  367 author: rohanrao / notebook: birdcall-eda-chirp-hoot-and-flutter
skipped
 -  368 author: rohitsingh9990 / notebook: eda-visualizations-simple-baseline
skipped
 -  369 author: shahules / notebook: bird-watch-complete-eda-fe
skipped
 -  370 author: shonenkov / notebook: competition-metrics
skipped
 -  371 author: shonenkov / notebook: sample-submission-using-custom-check
skipped
 -  372 author: tanulsingh077 / notebook: audio-albumentations-transform-your-audio
skipped
 -  373 author: tarunpaparaju / notebook: birdcall-identification-spectrogram-resnet
skipped
 -  374 author: ttahara / notebook: inference-birdsong-baseline-resnest50-fast
skipped
 -  375 author: ttahara / notebook: training-birdsong-bas

 -  517 author: bguberfain / notebook: just-showing-a-few-images
skipped
 -  518 author: bguberfain / notebook: naive-keras-cdiscount
skipped
 -  519 author: bguberfain / notebook: naive-statistics
skipped
 -  520 author: bguberfain / notebook: not-so-naive-way-to-convert-bson-to-files
skipped
 -  521 author: blazeka / notebook: multi-gpu-tensorflow-convnet-0-65
skipped
 -  522 author: blazeka / notebook: validate-download-with-sha256-hash
skipped
 -  523 author: cerebrium / notebook: multi-class-logistic-regression-using-tensor-flow
skipped
 -  524 author: ezietsman / notebook: inception-v3-finetune
skipped
 -  525 author: humananalog / notebook: keras-generator-for-reading-directly-from-bson
skipped
 -  526 author: inversion / notebook: processing-bson-files
skipped
 -  527 author: lamdang / notebook: fast-shuffle-bson-generator-for-keras
skipped
 -  528 author: mihaskalic / notebook: keras-xception-model-0-68-on-pl-weights
skipped
 -  529 author: mpekalski / notebook: convert-bson-t

...

 -  4398 author: prokaj / notebook: bert-joint-baseline-notebook
notebook not found
 -  4399 author: ragnar123 / notebook: exploratory-data-analysis-and-baseline
notebook not found
 -  4400 author: sakami / notebook: tfqa-pytorch-baseline
notebook not found
 -  4401 author: seesee / notebook: submit-full
notebook not found
 -  4402 author: xhlulu / notebook: tf-qa-jsonl-to-dataframe
notebook not found
 -  4403 author: yihdarshieh / notebook: inference-use-hugging-face-models
notebook not found
 -  4404 author: yutanakamura / notebook: nlp-express-0-data-loading-visualization
notebook not found
### 362 ../data/repositories/kaggle/competitions/c\text-normalization-challenge-english-language
 -  4405 author: allunia / notebook: eda-en-text-normalization
notebook not found
 -  4406 author: alphasis / notebook: bigdata-trick-or-treat-lb-0-9954
notebook not found
 -  4407 author: alvira12 / notebook: class-wise-processing-lb-0-992-new-dataset
notebook not found
 -  4408 author: amitabhac / n

skipped
 -  4507 author: wesamelshamy / notebook: trackml-problem-explanation-and-data-exploration
skipped
 -  4508 author: yuval6967 / notebook: 7th-place-clustering-extending-ml-merging-0-75
skipped
### 376 ../data/repositories/kaggle/competitions/c\tradeshift-text-classification
 -  4509 author: tarunaryyan / notebook: imputation-for-missing-values-in-features
skipped
### 377 ../data/repositories/kaggle/competitions/c\transfer-learning-on-stack-exchange-tags
 -  4510 author: akshatpathak / notebook: text-data-clustering
skipped
 -  4511 author: anokas / notebook: frequent-words-model-v2
skipped
 -  4512 author: charlescostello / notebook: transfer-learning-on-stack-exchange-tags
skipped
 -  4513 author: eliotbarr / notebook: word-clouds
skipped
 -  4514 author: katarz / notebook: tags-exploration
skipped
 -  4515 author: l3nnys / notebook: useful-text-preprocessing-on-the-datasets
skipped
 -  4516 author: mrtroll / notebook: analying-tfidf-biology-corpus
skipped
 -  4517 author: mrt

skipped
 -  4638 author: willieliao / notebook: et1-ridge3-med-adj
skipped
 -  4639 author: ymcdull / notebook: ridge-lb-0-0100659
skipped
 -  4640 author: ysidhu / notebook: two-sigma-portfolio-returns-eda
skipped
### 387 ../data/repositories/kaggle/competitions/c\two-sigma-financial-news
 -  4641 author: artgor / notebook: eda-feature-engineering-and-everything
skipped
 -  4642 author: ashishpatel26 / notebook: attension-layer-basic-for-nlp
skipped
 -  4643 author: ashishpatel26 / notebook: bird-eye-view-of-two-sigma-nn-approach
skipped
 -  4644 author: bguberfain / notebook: a-simple-model-using-the-market-and-news-data
skipped
 -  4645 author: chocozzz / notebook: two-sigma-news-simple-eda-prophet-nlp
skipped
 -  4646 author: christofhenkel / notebook: market-data-nn-baseline
skipped
 -  4647 author: danielson / notebook: cleaning-up-market-data-errors-and-stock-splits
skipped
 -  4648 author: dmdm02 / notebook: complete-eda-voting-lightgbm
skipped
 -  4649 author: dmitrypukhov / n

 -  4749 author: dishask99 / notebook: notebook8bb2428ec0
skipped
 -  4750 author: sandeepsingh3480 / notebook: notebook8ecd7cbceb
skipped
### 397 ../data/repositories/kaggle/competitions/c\web-traffic-time-series-forecasting
 -  4751 author: arjunsurendran / notebook: using-lstm-on-training-data
skipped
 -  4752 author: attollos / notebook: time-series-forecast-example-with-prophet
skipped
 -  4753 author: chechir / notebook: weekend-flag-median-with-wiggle
skipped
 -  4754 author: clustifier / notebook: weekend-weekdays
skipped
 -  4755 author: cpmpml / notebook: smape-weirdness
skipped
 -  4756 author: dextrousjinx / notebook: brief-insight-on-web-traffic-time-series
skipped
 -  4757 author: gvyshnya / notebook: parallel-operations-over-a-pandas-df
skipped
 -  4758 author: gvyshnya / notebook: prophet-class-wrapper
skipped
 -  4759 author: headsortails / notebook: wiki-traffic-forecast-exploration-wtf-eda
skipped
 -  4760 author: merckel / notebook: preliminary-investigation-holtwin

 -  4889 author: xchmiao / notebook: popcorn-rnn-model
skipped
 -  4890 author: yepp2411 / notebook: baseline-model-using-nn-for-movie-review
skipped
### 409 ../data/repositories/kaggle/competitions/c\worldcup2010
### 410 ../data/repositories/kaggle/competitions/c\worldcupconf
### 411 ../data/repositories/kaggle/competitions/c\yandex-personalized-web-search-challenge
### 412 ../data/repositories/kaggle/competitions/c\yelp-recruiting
### 413 ../data/repositories/kaggle/competitions/c\yelp-recsys-2013
### 414 ../data/repositories/kaggle/competitions/c\yelp-restaurant-photo-classification
 -  4891 author: anokas / notebook: patch-features-rfr
skipped
 -  4892 author: aparajit0511 / notebook: yelp-restaurant
skipped
 -  4893 author: ashbunny / notebook: kernel909af544ff
skipped
 -  4894 author: benhamner / notebook: sample-photos
skipped
 -  4895 author: dmytrolystopad / notebook: imagine-an-image1
skipped
 -  4896 author: enerrio / notebook: data-exploration-yelp-classification
skipped
 -

In [9]:
# scoring function to get a score between 0...1 for integer-values, 0.5 should be at ~100
def score(n, precision=3):
    if isinstance(n, int) or isinstance(i, float):
        return round(1-1/math.pow(1+n, 0.15), precision)
    else:
        try:
            n = int(n)
        except:
            return 0
        
    return round(1-1/math.pow(1+n, 0.15), precision)

for n in [0,1,10,25,50,100,1000,10000]:
    print(score(n))
    
print(score('3'))
print(score('a'))

0.0
0.099
0.302
0.387
0.446
0.5
0.645
0.749
0.188
0


In [80]:
# throw all parsed meta-data together in a single csv
# select only true ML cases

folder_base = '../data/repositories/kaggle/competitions/'
file_json = 'meta.json'
fp_csv = '../data/database/kaggle_competitions_01_original.csv'
fp_research = '../data/database/kaggle_competitions_02_research.csv'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0
j = 0

runtime_start = time.time()
df = pd.DataFrame()
df2 = pd.DataFrame()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', len(folders))
    for subfolder in subfolders:
        #print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,subfolder, 'notebooks/')
        i += 1
        
        if os.path.exists(path):
            projects = os.listdir(path)

            for author in projects:
                #print('author:', author)
                items = os.listdir(os.path.join(folder_base,folder,subfolder, 'notebooks/', author))

                for notebook in items:
                    j+=1
                    #print(' - ', j, 'author:', author, '/ notebook:', notebook)
                    fp_json = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_json)
                    
                    if os.path.isfile(fp_json):
                        data = load_data(fp_json, fromJson=True)
                        data['score_votes'] = score(data['votes'])
                        data['score_views'] = score(data['views'])
                        ml_score = 0
                        if len(data['ml_slugs']) > 0:
                            ml_score += 0.2
                        if len(data['ml_terms']) > 0:
                            ml_score += 0.3
                        if len(data['ml_libs']) > 0:
                            ml_score += 0.5
                        data['ml_detected'] = ml_score
                        
                        data['description'] = data['description'].replace('\n', ' ').replace('\r', '').replace('¶', '').strip()
                        
                        # date (ignoring GMT+x)
                        # Wed Dec 19 2018 14:42:40 GMT+0100 (Mitteleuropäische Normalzeit)
                        # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
                        date_time_str = data['date'].split('GMT')
                        date_time_str = date_time_str[0].strip()
                        date_time_obj = datetime.datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')

                        #print('Date:', date_time_obj.date())
                        #print('Time:', date_time_obj.time())
                        #print('Date-time:', date_time_obj)
                        data['date'] = date_time_obj
                        
                        # store only items with:
                        # - ml_score >= 0.5
                        # - are indicated as "best submission" # and 'Best Submission' in data['submission'] 
                        # - description > 1 word
                        words = data['description'].split(' ')
                        if ml_score > 0.5 and len(words) > 5:
                            df = df.append(data, ignore_index=True)
                        
                        else:
                            #if 'R' in data['type']:
                            df2 = df2.append(data, ignore_index=True)
                            
                    #if j % 100 == 0:
                    #    print('folder', i, '/ notebook', j)
                        
                if quit!=0 and j>quit:
                    break
            if quit!=0 and j>quit:
                break
    if quit!=0 and j>quit:
        break
        
# drop duplicates
df = df.drop_duplicates(['link'])

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', j, 'items')
print(df.shape)
print(df.head())
        
# drop columns
df.drop(columns=['author', 'submission'], inplace=True)
df2.drop(columns=['author', 'submission'], inplace=True)

df.to_csv(fp_csv, sep=';', index=False)
df2.to_csv(fp_research, sep=';', index=False)

folder: 1
subfolder: 1
runtime: 51.662 seconds for 5072 items
(1822, 22)
          author                date  \
0         a45632 2019-01-11 10:49:10   
1     amansohane 2018-12-24 16:27:39   
2  ashishpatel26 2019-01-04 08:43:54   
3  ashishpatel26 2018-12-27 10:13:57   
4  ashishpatel26 2018-12-19 14:42:40   

                                         description     license  \
0  Hybrid solution update using multiple sources....  Apache 2.0   
1  This is an attempt to solve difficulty 3. It b...  Apache 2.0   
2  What is Attention? Attention is simply a vecto...  Apache 2.0   
3  Beginner to Intermediate Natural Language Proc...  Apache 2.0   
4  This a fork of @opanichev 's great kernel: htt...  Apache 2.0   

                                                link  ml_detected  \
0  https://www.kaggle.com/a45632/classification-t...          1.0   
1  https://www.kaggle.com/amansohane/level-3-with...          1.0   
2  https://www.kaggle.com/ashishpatel26/attension...          1.0   
3